In [ ]:
import sys
sys.path
sys.path.append('../service/')

print(sys.executable)

In [ ]:
# standart libs
import random
import copy

# 3rd party libs
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import gempy as gp
from gempy.utils import section_utils

# local
import functions as fun

## Init the geo_model

In [ ]:
# instantiate the geo_model
geo_model = gp.create_model("BaseModel")

# defautl data
geo_model = gp.init_data(
    geo_model,
    extent=[0, 1, 0, 1, 0, 1],
    resolution=[5, 5, 5]
)

# compile theno function
gp.set_interpolation_data(
    geo_model,
    compile_theano=True,
    theano_optimizer='fast_run',
)

### Series

In [ ]:
# set some series
series_df = pd.DataFrame(columns=['name', 'isfault', 'order_series'])
series_df.loc[2] = { 'order_series': 2, 'name': 'Fault_Series', 'isfault': True }
series_df.loc[0] = { 'order_series': 0, 'name': 'Basement', 'isfault': False }
series_df.loc[1] = { 'order_series': 1, 'name': 'Strat_Series', 'isfault': False }

In [ ]:
# old series
geo_model.add_series(series_list=['S1', 'S2', 'S3'])

In [ ]:
fun.update_series(geo_model, series_df)
geo_model.series.df

### Surfaces

In [ ]:
surfaces_df = pd.DataFrame(columns=['name', 'serie', 'order_surface'])
surfaces_df.loc[0] = { 'name': 'Sandstone', 'serie': 'Strat_Series', 'order_surface': 0 }
surfaces_df.loc[1] = { 'name': 'Silt', 'serie': 'Strat_Series', 'order_surface': 1 }
surfaces_df.loc[2] = { 'name': 'Shale', 'serie': 'Strat_Series', 'order_surface': 2 }

In [ ]:
geo_model.add_surfaces(['Fake1', 'Fake2'])

In [ ]:
fun.update_surfaces(geo_model, surfaces_df)
geo_model.surfaces.df

### Input

In [ ]:
geo_model_extent = [0,2000,0,2000,0,2000]
section = {
    'p1': [0, 1000],
    'p2': [2000, 1000],
    'resolution': [200, 200]
}

series_df = pd.read_csv('./data/simple_fault_model_series.csv')
surfaces_df = pd.read_csv('./data/simple_fault_model_surfaces.csv')

surface_points_input_data = pd.read_csv('./data/simple_fault_model_points.csv')
orientaions_input_data = pd.read_csv('./data/simple_fault_model_orientations.csv')

# Format geological_input_data
surface_points_original_df = surface_points_input_data[['X', 'Y', 'Z', 'formation']]

# rename colums
surface_points_original_df.columns = ['X', 'Y', 'Z', 'surface']

# add distribution type and parameter
surface_points_original_df['param1'] = 10

# Orientaions
orientations_original_df = orientaions_input_data[['X', 'Y', 'Z', 'dip', 'azimuth', 'polarity', 'formation']]

In [ ]:
surfaces_df

In [ ]:
fun.set_congiguration(geo_model, geo_model_extent, section)
fun.update_series(geo_model, series_df)
fun.update_surfaces(geo_model, surfaces_df)
geo_model.set_surface_points(surface_points_original_df, update_surfaces=False)
geo_model.set_orientations(orientations_original_df, update_surfaces=False)
geo_model.update_to_interpolator()

In [ ]:
fun.check_setup_single_realization(geo_model)

In [ ]:
geo_model.surfaces

In [ ]:
# solution = gp.compute_model(model=geo_model, sort_surfaces=True)
# solution = gp.compute_model(model=geo_model, sort_surfaces=True)
solution = gp.compute_model(model=geo_model, sort_surfaces=False)
solution = gp.compute_model(model=geo_model, sort_surfaces=False)

In [ ]:
polygondict, cdict, extent = gp.utils.section_utils.get_polygon_dictionary(
    geo_model=geo_model,
    section_name="section"
)

In [ ]:
# Constants
n_realizations = 10

# Run realizations
list_section_data = fun.run_realizations(
    geo_model=geo_model,
    n_realizations=n_realizations,
    surface_points_original_df=surface_points_original_df,
    orientations_original_df=orientations_original_df,
    section=section
)

# Process output
section_data_stack, lithology_ids = fun.process_list_section_data(
    list_section_data=list_section_data)

# Count occurrences
count_array = fun.count_lithology_occurrences_over_realizations(
        section_data_stack=section_data_stack,
        lithology_ids=lithology_ids,
        section=section
)

# Calculate entrop
entropy_map = fun.calculate_information_entropy(
    count_array=count_array, n_realizations=n_realizations)

In [ ]:
plt.imshow(entropy_map[:,::-1].T, cmap='viridis')
plt.colorbar()
plt.show()